1. Access the first page of the product using requests

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = 'https://www.ceneo.pl/46475725/opinie-1'
response = requests.get(url)


In [ ]:
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")